<a href="https://colab.research.google.com/github/StanfordGeospatialCenter/SGC-Python-Geocoder/blob/working/CSV_to_Geocodes_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports & Job Parameters

These get adjusted based upon how many records you want per chunk, desired CRS, etc...

In [0]:
import requests
import json
import csv
import socket
import pprint
from datetime import datetime
from itertools import islice

#Input csv file
inCSV = open("testAddresses.csv")
# output spatial reference id 
outsrid = '4326'
# Total number of records to submit
sliceSize = 20
# Number of records to submit per GET request
chunkSize = 5

## Token request

In [35]:
## api-endpoint for requesting a token from locator.stanford.edu

tokenURL = "https://locator.stanford.edu/arcgis/tokens/generateToken"
payload = "username=XXXX&password=XXXX&client=requestip&expiration=86400&f=json"
headers = {
    'Content-Type': "application/x-www-form-urlencoded",
    'User-Agent': "PostmanRuntime/7.13.0",
    'Accept': "*/*",
    'Cache-Control': "no-cache",
    'Postman-Token': "2686259f-3e03-44d3-920e-abd43ae0c908,b4f42197-a2ae-4ef3-9dca-c0f5932bb21b",
    'Host': "locator.stanford.edu",
    'accept-encoding': "gzip, deflate",
    'content-length': "81",
    'Connection': "keep-alive",
    'cache-control': "no-cache"
    }
# request token from locator.stanford.edu/arcgis/tokens using REST POST
mytoken = requests.post( tokenURL, data=payload, headers=headers).json()['token']
print(mytoken)

MBDKV6_HesOnq0eO2yMjM4hOkHxF8cFOJyrh6d1MveGS1a3untpH9Rrt7EgVJejP


## Parameters for the geocoding REST POST request

In [0]:
# api-endpoint for geocoding US Street Addresses with locator.stanford.edu

url = 'https://locator.stanford.edu/arcgis/rest/services/geocode/USA_StreetAddress/GeocodeServer/geocodeAddresses'

#output format
format = 'json'

# header
header = {'Content-Type': 'application/x-www-form-urlencoded'}

# params data to be sent to api 
params ={'outSR':outsrid,'f':format,'token':mytoken}

## Make a new N record CSV from the oneMillionAddresses.csv then chunk & submit geocodes to locator.stanford.edu, then write response to new csv

In [38]:
#Job startTime
jobStartTime = datetime.now()
print(jobStartTime)

slice = islice(inCSV,sliceSize)
reader = csv.reader(slice)

with open('slicedAddresses.csv', 'w',newline='') as outCSV:
    writer = csv.writer(outCSV)
    for row in reader:
        writer.writerow(row)# Path to the sliced csv file
csvPath = 'slicedAddresses.csv'

# The records number to start the next chunk at
chunkMarker = 0

# Open a file to write the resulting geocodes to
outCSV = open('outCSV.csv','w',newline='')

#Write the headers to the opened outCSV.csv 
geocodeWriter = csv.DictWriter(outCSV,fieldnames=('LangCode', 'Score', 'Side', 'Status', 'StAddr', 'Subregion', 'Distance', 'AddNumTo', 'StDir', 'Region', 'StType', 'Match_addr', 'Postal', 'AddNum', 'StPreType', 'StName', 'City', 'Country', 'X', 'Addr_type', 'AddNumFrom', 'StPreDir', 'ResultID', 'Y'))
geocodeWriter.writeheader()

#Open the input csv and count the rows minus the header for a number of data rows
with open(csvPath) as f:
    numRows = (len(list(f))-1)
    
# calculate the number of chunks based upon the numRows and chunkSize
    for i in range(int((numRows-(numRows%chunkSize))/chunkSize)+1):
#Print the start time for the chunk
        #chunkStartTime = datetime.now()
        #print("Chunk #"+ str(i) + " start time:"+str(chunkStartTime))
#Open the sliced csv
        with open(csvPath) as csvFile:
#Create a csv DictReader from the csvFile
            csvReader = csv.DictReader(csvFile)
#Create a list from the DictReader
            csvRows = list(csvReader)
#Nest each record under an attribute oblect, iterating through the csvRows
            attribute = [{'attributes': row }for row in csvRows[chunkMarker:chunkMarker+chunkSize]]
#Dump the result to json
            addresses = json.dumps({'records':attribute}) 
            #print("Output",addresses)
#Submit the chunk for geocoding
            r = requests.post(url, headers = header, data = { 'addresses':addresses,'outSR':outsrid,'f':format,'token':mytoken})
            parsedJson = json.loads(r.text)
            print("Results",r.text)
#Get the current chunk length to evaluate
            #print("this chunk length: "str(len(list(parsedJson['locations']))))
#iterate over the result, writing each attribute record to a new csv row
#Counter for iterating over the result records, based upon the chunkSize
            for c in range(chunkSize):
                try:
                
#Open the outCSV again in append mode
                    outCSV = open('outCSV.csv','a',newline='')
    #Create a DictWriter to write the rows
                    geocodeWriter = csv.DictWriter(outCSV,fieldnames=('LangCode', 'Score', 'Side', 'Status', 'StAddr', 'Subregion', 'Distance', 'AddNumTo', 'StDir', 'Region', 'StType', 'Match_addr', 'Postal', 'AddNum', 'StPreType', 'StName', 'City', 'Country', 'X', 'Addr_type', 'AddNumFrom', 'StPreDir', 'ResultID', 'Y'))
    #Parse the current records for element [record] 
                    #print is_json("{}")
                    jsonRows = parsedJson['locations'][c]['attributes']                
    #Write the values to a row
                    geocodeWriter.writerow(jsonRows)
                except IndexError:
#                     print("End of file")
                    break
#Print the end time for the chunk and the processing time
            #chunkEndTime = datetime.now()
            #print("Chunk #"+str(i)+" end time: "+str(chunkEndTime))
            #print("Chunk #"+str(i)+" total time: "+str(chunkEndTime - chunkStartTime))
#Calculate the next chunk's starting point 
            chunkMarker = chunkMarker+chunkSize
            print(str(chunkMarker)+" records geocoded",end='\r')
outCSV.close()

#Reporting & logging
#Get some data
hostname = socket.gethostname()    
IPAddr = socket.gethostbyname(hostname)    
jobEndTime = datetime.now()
jobTotalTime = jobEndTime-jobStartTime
jobTotalSeconds = jobTotalTime.total_seconds()
recordsPerSecond = (sliceSize/jobTotalSeconds)
millionRecordTime = (((1000000/sliceSize)*jobTotalSeconds)/3600)
#Log to a csv
with open('geocodeLogs.csv', 'a', newline='') as csvfile:
    fieldnames = ['hostname','IPAddr','jobStartTime','jobEndTime', 'jobTotalTime', 'sliceSize','chunkSize','recordsPerMinute','recordsPerHour','millionRecordTime']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    #writer.writeheader()
    writer.writerow({'hostname':hostname,'IPAddr':IPAddr,'jobStartTime':jobStartTime,'jobEndTime':jobEndTime, 'jobTotalTime':jobTotalTime, 'sliceSize':sliceSize,'chunkSize':chunkSize,'recordsPerMinute':recordsPerSecond*60,'recordsPerHour':recordsPerSecond*3600,'millionRecordTime':millionRecordTime})
    print(str(chunkMarker)+" records geocoded and finished in: "+str(jobTotalSeconds)+ " seconds")

2020-01-30 20:08:16.870912
Results {"spatialReference":{"wkid":4326,"latestWkid":4326},"locations":[{"address":"","location":{"x":"NaN","y":"NaN"},"score":0,"attributes":{"ResultID":-1,"Status":"U","Score":0,"Match_addr":"","Addr_type":"","AddNum":"","AddNumFrom":"","AddNumTo":"","Side":"","StPreDir":"","StPreType":"","StName":"","StType":"","StDir":"","StAddr":"","City":"","Subregion":"","Region":"","Postal":"","Country":"","LangCode":"","Distance":0,"X":0,"Y":0}},{"address":"","location":{"x":"NaN","y":"NaN"},"score":0,"attributes":{"ResultID":-1,"Status":"U","Score":0,"Match_addr":"","Addr_type":"","AddNum":"","AddNumFrom":"","AddNumTo":"","Side":"","StPreDir":"","StPreType":"","StName":"","StType":"","StDir":"","StAddr":"","City":"","Subregion":"","Region":"","Postal":"","Country":"","LangCode":"","Distance":0,"X":0,"Y":0}},{"address":"","location":{"x":"NaN","y":"NaN"},"score":0,"attributes":{"ResultID":-1,"Status":"U","Score":0,"Match_addr":"","Addr_type":"","AddNum":"","AddNumFr